## Иерархическая индексация (heirarchical indexing)
    Многомерные данные могут храниться путем использования объектов Panel - 3D, и Panel4D - для 4D или
    Иерархической индексации (или мультииндекс multi-indexing) для N-мерных структур.

In [1]:
import numpy as np
import pandas as pd

### Мультииндексированный объект Series

In [2]:
index = [
    ('c', 2000), ('c', 2010),
    ('n', 2000), ('n', 2010),
    ('t', 2000), ('t', 2010)
]
populations = [3387, 3725,
              1897, 1937,
              2085, 2514]

In [3]:
pop = pd.Series(populations, index=index)

In [4]:
pop
# это еще не мультииндекс, это картежи-ключи

(c, 2000)    3387
(c, 2010)    3725
(n, 2000)    1897
(n, 2010)    1937
(t, 2000)    2085
(t, 2010)    2514
dtype: int64

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('c', 2000),
            ('c', 2010),
            ('n', 2000),
            ('n', 2010),
            ('t', 2000),
            ('t', 2010)],
           )

In [6]:
pop = pop.reindex(index)
# вот это мультииндекс

In [7]:
pop

c  2000    3387
   2010    3725
n  2000    1897
   2010    1937
t  2000    2085
   2010    2514
dtype: int64

In [8]:
# мультииндекс любит срезы:
pop[:, 2010]

c    3725
n    1937
t    2514
dtype: int64

#### Мультииндекс как доп. измерение:

In [9]:
# мультииндексный Series м.б. преобразован в обычный DataFrame
pop_df = pop.unstack()

In [10]:
pop_df

,2000,2010
c,3387,3725
n,1897,1937
t,2085,2514


In [11]:
# и наоборот, DataFrame м.б. превращен в мультииндексный Series
pop_df.stack()

c  2000    3387
   2010    3725
n  2000    1897
   2010    1937
t  2000    2085
   2010    2514
dtype: int64

In [13]:
# можно добавать доп столбец
pop_df = pd.DataFrame({'total': pop,
                      'under18': [926, 928,
                                 468, 431,
                                 590, 687]})
pop_df

total  under18
c 2000   3387      926
  2010   3725      928
n 2000   1897      468
  2010   1937      431
t 2000   2085      590
  2010   2514      687

In [14]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18
# сакой способ позволяет легко манипулировать многомерными данными

c  2000    0.273398
   2010    0.249128
n  2000    0.246705
   2010    0.222509
t  2000    0.282974
   2010    0.273270
dtype: float64

In [15]:
f_u18.unstack()

,2000,2010
c,0.273398,0.249128
n,0.246705,0.222509
t,0.282974,0.273270


### Методы создания мультииндексов:
    Наиболее простой метод создания мультииндекса - передать в конструктор список из двух и более индексных массивов (стр. 168)

In [16]:
df = pd.DataFrame(np.random.rand(4,2),
                 index=[['a', 'a', 'b', 'b'],
                         [1, 2, 1, 2]],
                 columns=['data1', 'data2'])
df

data1     data2
a 1  0.293791  0.134242
  2  0.543200  0.989282
b 1  0.357041  0.652495
  2  0.770514  0.093139

In [17]:
# Если передавать словарь, с кортежами в качестве ключей Pandas автоматически распознает мультииндекс:
data = {
    ('c', 2000): 3387,
    ('c', 2010): 3725,
    ('n', 2000): 2085,
    ('n', 2010): 2514,
    ('t', 2000): 1897,
    ('t', 2010): 1937,
}

In [18]:
pd.Series(data)

c  2000    3387
   2010    3725
n  2000    2085
   2010    2514
t  2000    1897
   2010    1937
dtype: int64

### Явные конструкторы для MultiIndex